In [29]:
from os.path import expanduser, join
from pandas import read_pickle
target_folder = expanduser('~/Experiments/spatiotemporal')
nyc_traffic_injury_with_period_table = read_pickle(join(
    target_folder, 'nyc-traffic-injury-with-period.pkl'))
nyc_traffic_injury_with_period_table[:3]

,Longitude,Latitude,Year,Month,Total,Pedestrian,Bike,Vehicle
Period,,,,,,,,
2009-01,-73.791745,40.725789,2009,1,2,1,0,1
2009-01,-73.882429,40.844981,2009,1,1,1,0,0
2009-01,-73.979058,40.744444,2009,1,2,2,0,0


In [30]:
nyc_traffic_injury_with_period_table.reset_index(inplace=True)
nyc_traffic_injury_with_period_table[:3]

,Period,Longitude,Latitude,Year,Month,Total,Pedestrian,Bike,Vehicle
0,2009-01,-73.791745,40.725789,2009,1,2,1,0,1
1,2009-01,-73.882429,40.844981,2009,1,1,1,0,0
2,2009-01,-73.979058,40.744444,2009,1,2,2,0,0


In [31]:
grouped_by_location = nyc_traffic_injury_with_period_table.groupby(['Longitude', 'Latitude'])
indices = grouped_by_location.groups[(-73.95900740013359, 40.582838127171144)]
indices

[2154, 92891, 95707, 160398]

In [40]:
# Select locations with repeat incidents
bad_location_table = grouped_by_location.filter(lambda x: len(x) > 2)
grouped_by_location = bad_location_table.groupby(['Longitude', 'Latitude'])
indices = grouped_by_location.groups[(-73.95900740013359, 40.582838127171144)]
indices

[2154, 92891, 95707, 160398]

In [64]:
local_injury_table = bad_location_table.ix[indices][['Period', 'Total', 'Pedestrian', 'Bike', 'Vehicle']]
local_injury_table

,Period,Total,Pedestrian,Bike,Vehicle
2154,2009-01,1,0,0,1
92891,2009-05,1,1,0,0
95707,2010-05,1,1,0,0
160398,2011-09,1,0,0,1


In [71]:
import numpy as np
resampled_local_injury_table = local_injury_table.set_index('Period').resample('M', np.sum).fillna(0).reset_index()
resampled_local_injury_table[:5]

,Period,Total,Pedestrian,Bike,Vehicle
0,2009-01,1,0,0,1
1,2009-02,0,0,0,0
2,2009-03,0,0,0,0
3,2009-04,0,0,0,0
4,2009-05,1,1,0,0


In [76]:
xs = [x.to_timestamp().value for x in resampled_local_injury_table['Period']]
xs[:5]

[1230768000000000000,
 1233446400000000000,
 1235865600000000000,
 1238544000000000000,
 1241136000000000000]

In [77]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit([[x] for x in xs], resampled_local_injury_table['Total'].values)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [82]:
from pandas import Period
model.predict(Period(year=2015, month=12, freq='M').to_timestamp().value)

array([-0.14804261])